In [ ]:
# Setup and Imports
import requests
import json
import time
import random
import os
from pathlib import Path
from typing import Dict, List, Any, Optional
import pandas as pd
from IPython.display import display, HTML

# Configuration
API_BASE_URL = "http://localhost:8000"
OLLAMA_API_BASE = "http://localhost:11000"  # From .env
MODEL = "qwen3:latest"
SCHEMA_NAME = "translate"
WIKI_DIR = Path("wiki_zh")
NUM_SAMPLES = 10
POLL_INTERVAL = 2  # seconds
MAX_WAIT = 300  # seconds

print(f"🔧 Configuration:")
print(f"   API Base: {API_BASE_URL}")
print(f"   Ollama Base: {OLLAMA_API_BASE}")
print(f"   Model: {MODEL}")
print(f"   Wiki Directory: {WIKI_DIR}")
print(f"   Samples: {NUM_SAMPLES}")
print(f"\n✓ Setup complete!")

In [ ]:
# Helper Functions

def check_api_health() -> bool:
    """Check if API is running and healthy."""
    try:
        response = requests.get(f"{API_BASE_URL}/health", timeout=5)
        response.raise_for_status()
        data = response.json()
        print(f"✓ API Status: {data.get('status')}")
        print(f"  Active Agent: {data.get('message')}")
        return data.get('status') == 'healthy'
    except Exception as e:
        print(f"✗ API health check failed: {e}")
        return False


def translate_with_model(text: str, model_name: str = MODEL) -> Optional[Dict[str, Any]]:
    """Submit translation job and poll for results."""
    try:
        # Submit async translation job
        response = requests.post(
            f"{API_BASE_URL}/translate/async",
            json={
                "text": text,
                "schema_name": SCHEMA_NAME,
                "model_override": model_name
            },
            timeout=10
        )
        response.raise_for_status()
        job_data = response.json()
        job_id = job_data.get('job_id')
        
        if not job_id:
            print(f"   ✗ No job_id returned")
            return None
        
        print(f"   Job ID: {job_id}")
        
        # Poll for completion
        start_time = time.time()
        while time.time() - start_time < MAX_WAIT:
            status_response = requests.get(
                f"{API_BASE_URL}/translate/status/{job_id}",
                timeout=10
            )
            status_response.raise_for_status()
            status_data = status_response.json()
            status = status_data.get('status')
            
            if status == 'completed':
                elapsed = time.time() - start_time
                print(f"   ✓ Completed in {elapsed:.1f}s")
                return {
                    'result': status_data.get('result', {}),
                    'elapsed_time': elapsed
                }
            elif status == 'failed':
                error = status_data.get('error', 'Unknown error')
                print(f"   ✗ Translation failed: {error}")
                return None
            
            time.sleep(POLL_INTERVAL)
        
        print(f"   ✗ Translation timed out after {MAX_WAIT}s")
        return None
        
    except Exception as e:
        print(f"   ✗ Translation error: {e}")
        return None


def extract_translation_text(result: Dict[str, Any], model_key: str) -> str:
    """Extract translated text from API result."""
    if not result:
        return ""
    
    result_data = result.get('result', {})
    translations = result_data.get('translations', {})
    model_data = translations.get(model_key, {})
    
    translation = model_data.get('translated_text') or \
                  model_data.get('translation') or \
                  model_data.get('text', '')
    
    return translation.strip() if translation else ""


# Verify API connectivity
print("\n🔍 Checking API health...")
if check_api_health():
    print("\n✓ API is ready!")
else:
    print("\n⚠️  API is not accessible. Please ensure:")
    print("   1. API server is running: python main.py")
    print(f"   2. API is accessible at {API_BASE_URL}")

In [ ]:
# Scan wiki_zh directory and collect file paths

print(f"🔍 Scanning {WIKI_DIR} for JSON files...\n")

if not WIKI_DIR.exists():
    print(f"❌ Directory {WIKI_DIR} does not exist!")
    wiki_files = []
else:
    # Recursively find all files in wiki_zh
    wiki_files = []
    for root, dirs, files in os.walk(WIKI_DIR):
        for file in files:
            file_path = Path(root) / file
            # Check if it looks like a data file (not hidden, reasonable size)
            if not file.startswith('.') and file_path.stat().st_size > 100:
                wiki_files.append(file_path)
    
    print(f"✓ Found {len(wiki_files)} potential data files")
    
    # Show some examples
    if wiki_files:
        print(f"\nSample files:")
        for f in wiki_files[:5]:
            print(f"   {f}")

In [ ]:
# Extract random entries from files

print(f"\n📝 Extracting {NUM_SAMPLES} random entries...\n")

wiki_entries = []

if not wiki_files:
    print("❌ No wiki files found!")
else:
    # Try to collect entries from random files
    random.shuffle(wiki_files)
    
    for file_path in wiki_files:
        if len(wiki_entries) >= NUM_SAMPLES:
            break
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                # Read all lines and parse as JSON
                lines = f.readlines()
                
                # Try to parse each line as JSON
                for line in lines:
                    line = line.strip()
                    if not line:
                        continue
                    
                    try:
                        entry = json.loads(line)
                        
                        # Check if entry has required fields
                        if 'text' in entry and 'title' in entry:
                            text = entry['text'].strip()
                            
                            # Filter out very short or very long texts
                            if 50 < len(text) < 2000:
                                wiki_entries.append({
                                    'id': entry.get('id', 'unknown'),
                                    'title': entry.get('title', 'Untitled'),
                                    'url': entry.get('url', ''),
                                    'text': text,
                                    'source_file': str(file_path)
                                })
                                
                                if len(wiki_entries) >= NUM_SAMPLES:
                                    break
                    except json.JSONDecodeError:
                        continue
        
        except Exception as e:
            print(f"   ⚠️  Error reading {file_path}: {e}")
            continue
    
    # Shuffle to randomize
    random.shuffle(wiki_entries)
    wiki_entries = wiki_entries[:NUM_SAMPLES]
    
    print(f"✓ Collected {len(wiki_entries)} entries for translation\n")
    
    # Display sample entries
    if wiki_entries:
        print("Sample entries:")
        for i, entry in enumerate(wiki_entries[:3], 1):
            print(f"\n{i}. {entry['title']}")
            print(f"   ID: {entry['id']}")
            print(f"   Text length: {len(entry['text'])} chars")
            print(f"   Preview: {entry['text'][:100]}...")

In [ ]:
# Main Translation Loop

print("\n" + "="*80)
print(f"🚀 STARTING TRANSLATION TEST WITH {MODEL}")
print("="*80)

results = []

if not wiki_entries:
    print("\n❌ No wiki entries to translate!")
else:
    for idx, entry in enumerate(wiki_entries, 1):
        print(f"\n{'='*80}")
        print(f"📖 Entry {idx}/{len(wiki_entries)}: {entry['title']}")
        print(f"{'='*80}")
        print(f"ID: {entry['id']}")
        print(f"Text: {entry['text'][:200]}...\n")
        
        # Submit translation
        print(f"🤖 Translating with {MODEL}...")
        result = translate_with_model(entry['text'], MODEL)
        
        if result:
            translation = extract_translation_text(result, MODEL)
            elapsed_time = result.get('elapsed_time', 0)
            
            result_entry = {
                'index': idx,
                'id': entry['id'],
                'title': entry['title'],
                'url': entry['url'],
                'source_file': entry['source_file'],
                'original_text': entry['text'],
                'original_length': len(entry['text']),
                'translation': translation,
                'translation_length': len(translation),
                'elapsed_time': elapsed_time,
                'chars_per_second': len(entry['text']) / elapsed_time if elapsed_time > 0 else 0,
                'success': True
            }
            
            results.append(result_entry)
            
            print(f"\n📄 Translation:")
            print(f"   {translation[:300]}...")
            print(f"\n⏱️  Time: {elapsed_time:.1f}s ({result_entry['chars_per_second']:.1f} chars/s)")
        else:
            result_entry = {
                'index': idx,
                'id': entry['id'],
                'title': entry['title'],
                'url': entry['url'],
                'source_file': entry['source_file'],
                'original_text': entry['text'],
                'original_length': len(entry['text']),
                'translation': '',
                'translation_length': 0,
                'elapsed_time': 0,
                'chars_per_second': 0,
                'success': False
            }
            results.append(result_entry)
            print(f"\n❌ Translation failed")
        
        # Brief pause between translations
        time.sleep(1)

print(f"\n\n{'='*80}")
print(f"✅ TRANSLATION TEST COMPLETE")
print(f"{'='*80}")
print(f"Total entries: {len(wiki_entries)}")
print(f"Successful translations: {sum(1 for r in results if r['success'])}")
print(f"Failed translations: {sum(1 for r in results if not r['success'])}")

In [ ]:
# Create Results DataFrame

if results:
    results_df = pd.DataFrame(results)
    
    print(f"\n📊 Results DataFrame:")
    print(f"   Shape: {results_df.shape}")
    print(f"   Successful: {results_df['success'].sum()}")
    
    # Display summary statistics
    print(f"\n📈 Performance Statistics:")
    successful = results_df[results_df['success']]
    
    if len(successful) > 0:
        print(f"   Average time: {successful['elapsed_time'].mean():.2f}s")
        print(f"   Median time: {successful['elapsed_time'].median():.2f}s")
        print(f"   Average speed: {successful['chars_per_second'].mean():.1f} chars/s")
        print(f"   Total characters translated: {successful['original_length'].sum():,}")
        print(f"   Total time: {successful['elapsed_time'].sum():.1f}s")
    
    # Display summary table
    print(f"\n📋 Summary Table:")
    display(results_df[['index', 'title', 'original_length', 'translation_length', 
                        'elapsed_time', 'chars_per_second', 'success']])
else:
    print("\n❌ No results to display")

In [ ]:
# Detailed Translation Comparison

if results:
    print("\n" + "="*80)
    print("📝 DETAILED TRANSLATION REVIEW")
    print("="*80)
    
    for idx, row in results_df.iterrows():
        print(f"\n{'='*80}")
        print(f"Entry {row['index']}: {row['title']}")
        print(f"{'='*80}")
        
        print(f"\n🔗 URL: {row['url']}")
        print(f"📁 Source: {row['source_file']}")
        print(f"🆔 ID: {row['id']}")
        
        print(f"\n📖 Original Text ({row['original_length']} chars):")
        print(f"   {row['original_text'][:400]}..." if len(row['original_text']) > 400 else f"   {row['original_text']}")
        
        if row['success']:
            print(f"\n🌍 Translation ({row['translation_length']} chars):")
            print(f"   {row['translation'][:400]}..." if len(row['translation']) > 400 else f"   {row['translation']}")
            
            print(f"\n⏱️  Performance:")
            print(f"   Time: {row['elapsed_time']:.2f}s")
            print(f"   Speed: {row['chars_per_second']:.1f} chars/s")
            print(f"   Length ratio: {row['translation_length']/row['original_length']:.2f}")
        else:
            print(f"\n❌ Translation failed")
        
        print()
else:
    print("\n❌ No results to display")

In [ ]:
# Export Results

if results:
    print("\n💾 Exporting results...\n")
    
    # Export to CSV
    csv_path = f"wiki_translation_results_{MODEL.replace(':', '_')}.csv"
    results_df.to_csv(csv_path, index=False)
    print(f"   ✓ CSV exported: {csv_path}")
    
    # Export to JSON (more detailed)
    json_path = f"wiki_translation_results_{MODEL.replace(':', '_')}.json"
    results_df.to_json(json_path, orient='records', indent=2, force_ascii=False)
    print(f"   ✓ JSON exported: {json_path}")
    
    print("\n✅ All results exported successfully!")
else:
    print("\n❌ No results to export")

In [ ]:
# Final Summary

print("\n" + "="*80)
print("✅ WIKI TRANSLATION TEST COMPLETE")
print("="*80)

if results:
    successful = results_df[results_df['success']]
    
    print(f"\n📊 Summary:")
    print(f"   • Total entries: {len(results_df)}")
    print(f"   • Successful: {len(successful)}")
    print(f"   • Failed: {len(results_df) - len(successful)}")
    print(f"   • Model: {MODEL}")
    
    if len(successful) > 0:
        print(f"\n⚡ Performance:")
        print(f"   • Average time: {successful['elapsed_time'].mean():.2f}s")
        print(f"   • Average speed: {successful['chars_per_second'].mean():.1f} chars/s")
        print(f"   • Total characters: {successful['original_length'].sum():,}")
        print(f"   • Total time: {successful['elapsed_time'].sum():.1f}s")
        
        avg_ratio = (successful['translation_length'] / successful['original_length']).mean()
        print(f"   • Avg translation ratio: {avg_ratio:.2f}x")
    
    print(f"\n📁 Output files:")
    print(f"   • {csv_path}")
    print(f"   • {json_path}")
    
    print(f"\n💡 Next steps:")
    print(f"   1. Review translations for quality and accuracy")
    print(f"   2. Compare with other models using the model comparison notebook")
    print(f"   3. Analyze translation patterns and common issues")
    print(f"   4. Test with more samples for comprehensive evaluation")
else:
    print(f"\n⚠️  No translations completed.")
    print(f"\n🔍 Troubleshooting:")
    print(f"   1. Verify API is running: {API_BASE_URL}")
    print(f"   2. Check Ollama is running: {OLLAMA_API_BASE}")
    print(f"   3. Ensure model is installed: ollama pull {MODEL}")
    print(f"   4. Verify wiki_zh directory exists and contains data")

print("\n" + "="*80)